In [1]:
import pandas as pd
import time

In [2]:
transcation= pd.read_excel('/Users/mac/Desktop/数据科学导论作业/ch2/Sample - Superstore.xls', sheetname='Orders', index_col='Row ID')
transcation = transcation[['Order ID','Product ID']]

/Users/mac/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [4]:
def sum_data(df):
    Product_X = pd.DataFrame(df.groupby('Product ID').size(),columns = ['n_X']).reset_index()
    Product_Y = pd.DataFrame(df.groupby('Product ID').size(),columns = ['n_Y']).reset_index()
    Product_X.columns = ['XID','n_X']
    Product_Y.columns = ['YID','n_Y']
    #make two dataframe contains the count of appearance in each OrderID for each product and prepare for the later merge
    transaction = len(df.groupby('Order ID').count())
    #count the number of orders
    df1=df
    df_m = pd.merge(df1,df,on=['Order ID'])
    df_m.columns = ['Order ID','XID','YID']
    df_m = df_m.drop(df_m[df_m['XID']==df_m['YID']].index.values)
    #outer join two dataframe(they are the same), to get the pairs based on the orderid and drop the paris of same products
    cross_count = pd.DataFrame(df_m.groupby(['XID','YID']).size(),columns = ['n_XY']).reset_index()
    #count the number of each pairs 
    sum_data = pd.merge(cross_count,Product_X,on=['XID'])
    sum_data = pd.merge(sum_data,Product_Y,on=['YID'])
    #merge the data with the first two dataframe to get the number of XID&YID for the later calculation
    sum_data['n_transaction']=transaction
    sum_data['support']=sum_data['n_X']/sum_data['n_transaction']
    sum_data['confidence']=sum_data['n_XY']/sum_data['n_X']
    sum_data['lift']=sum_data['confidence']/sum_data['n_Y']
    return sum_data,transaction,len(Product_X)

In [8]:
t0=time.time()
sample_result,sample_transaction,sample_Goods= sum_data(sample)
print('execution time:' ,time.time()-t0)

execution time: 0.040540218353271484


In [16]:
t0 = time.time()
sum_result,n_transaction,n_Goods = sum_data(transcation)
print('n_transaction',n_transaction)
print('n_Goods',n_Goods)
print('npairs',len(sum_result))
print('total execution time:',time.time()-t0)

n_transaction 5009
n_Goods 1862
npairs 19786
total execution time: 0.08544421195983887


In [17]:
with pd.ExcelWriter('/Users/mac/Desktop/数据科学导论作业/ch2/sum_data.xlsx') as writer:
    sample_result.to_excel(writer,sheet_name='sheet1')
    sum_result.to_excel(writer,sheet_name='sheet2')